In [1]:
import xarray as xr
import numpy as np
from my_funs import est_trend

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/working/"


Estimating trend using theilsen method and the significance of trend using mannkendall method 

In [41]:
# Growing season mean
print("Estimating trend for growing season mean")
lai_growing_mean = xr.open_dataarray(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_mean.nc"
)
est_trend_lai_growing_mean = est_trend(lai_growing_mean, "theilsen")
est_trend_lai_growing_mean.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_growing_trend_mean.nc"
)


Estimating trend for growing season mean


In [2]:
# Monthly mean
lai_monthly_mean = xr.open_dataset(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_mean.nc"
)["LAI"]

monthly_trends_datasets = []
for k in np.arange(5, 11):
    print(k)
    tmp = lai_monthly_mean.where(lai_monthly_mean.time.dt.month == k, drop=True)
    est_trend_tmp = est_trend(tmp, "theilsen")
    monthly_trends_datasets.append(est_trend_tmp)
est_trend_lai_monthly_mean = xr.concat(
    monthly_trends_datasets, dim="months"
).assign_coords({"months": np.arange(5, 11)})
est_trend_lai_monthly_mean.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_monthly_trend_mean.nc"
)


5
6
7
8
9
10


<xarray.Dataset>
Dimensions:      (latitude: 448, longitude: 1348, months: 6)
Coordinates:
  * latitude     (latitude) float32 74.53 74.47 74.43 ... 52.27 52.22 52.17
  * longitude    (longitude) float32 -168.1 -168.0 -168.0 ... -100.8 -100.7
    spatial_ref  int64 0
  * months       (months) int64 5 6 7 8 9 10
Data variables:
    trend        (months, latitude, longitude) float64 nan nan nan ... nan nan
    p_value      (months, latitude, longitude) float64 nan nan nan ... nan nan
    h            (months, latitude, longitude) float64 nan nan nan ... nan nan
Attributes:
    trend:    The estimated trend using Theilsen method
    p:        Estimated p_value using Mann-Kendall
    h:        1 if trend is significant (p<0.05), 0 otherwise